&nbsp;
# **Toilet & Bench Coverage***
###### *inside an area

&nbsp;
# Imports and credentials
For this recipe :) we will need:

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [146]:
import pandas as pd, numpy as np

#geo libraries
import geopandas as gpd
from shapely.geometry import Point, Polygon
#optional speed-ups
from shapely import speedups 
speedups.enable() 

import h3
import overpass

#graphics
import pydeck as pdk

#a mapbox api key for tiles (get your own at mapbox.com)
with open('./.to_ignore/keys/v1_mapbox.key', 'r') as f:
    mapbox_key= f.read()[:-1]
#and a choice of mapbox tiles
map_style='mapbox://styles/mapbox/light-v10'

&nbsp;
## OpenStreetmap's Data + Overpass API
&nbsp;

#### Below is a couple simple method that build geoDataFrames from:
    * Querying OSM for all nodes/ways
    * Tagged as "amenity"with "bench" or "toilets" as a value
    * In a geographical area given by name and administrative level (see [here](https://www.openstreetmap.org/relation/52822) )

&nbsp;

In [2]:
def fetch_amenity_gdf(amenity_types, area_name, adm_lvl, timeout_sec=90):
    #exports a shapefile with OSM's response to a request of data for
    #amenities of amenity_type in area_name / adm_lvl.
    query=f'( area[admin_level={adm_lvl}]["name"="{area_name}"]; ) ->.a; \n('
    node_req='\nnode["amenity"="{0}"](area.a);\nway["amenity"="{0}"](area.a);'
    for x in amenity_types:
        query=query+node_req.format(x)
    query=query+'\n);'
    print(query)
    api = overpass.API(debug=False, timeout=timeout_sec)
    try:
        result = api.Get(query, responseformat="geojson")
        result = gpd.GeoDataFrame.from_features(result['features'])
        result.to_file('./.to_ignore/out/'+f'gdf_amenities_{area_name}_{adm_lvl}') 
        print('Downloaded to ./.to_ignore/out/'+f'gdf_amenities_{area_name}_{adm_lvl}')
    except Exception as HELL:
        print(HELL)   

&nbsp;

# Fetch & clean geoDataFrame from OSM

&nbsp;

#### With features on toilets and benches

&nbsp;

Avoid querying if file already exists

In [3]:
area='Sverige'#'Göteborgs Stad'
lvl='2'#'7' 

In [4]:
try:
    gdf_amenity=gpd.read_file('./.to_ignore/out/'+f'gdf_amenities_{area}_{lvl}')
except:
    print('file_not_found, fetching amenities from OSM')
    fetch_amenity_gdf(["toilets","bench"], area,lvl)
    gdf_amenity=gpd.read_file('./.to_ignore/out/'+f'gdf_amenities_{area}_{lvl}')
gdf_amenity.dropna(subset=['geometry'], inplace=True) #remove nans on geometry
gdf_amenity = gdf_amenity.set_crs(epsg=4326) # this is the CRS in OSM

&nbsp;

## What features do we have here?

In [5]:
print('Features: '+','.join(gdf_amenity.keys()))

Features: amenity,created_by,wheelchair,backrest,source,material,fee,layer,access,table,tourism,seamark_sm,seamark_ty,fixme,name,descriptio,note,operator,emergency,indoor,contact_ph,opening_ho,drinking_w,viewpoint,colour,fee_price,toilets_di,unisex,descript_1,women,check_date,direction,seats,amount,changing_t,capacity,capacity_d,level,snowmobile,female,sanitary_d,public_tra,shelter,toilets_wh,bench_coun,leisure,toilets_po,supervised,network,toilets_ha,ele,route_ref,ref,building,survey_dat,artwork,diaper,male,toilets_ac,designatio,natural,note_sv,highway,fireplace,bench,toilets__1,toilets_pa,water,descript_2,addr_city,addr_count,addr_house,addr_stree,wheelcha_1,charge,history,covered,shop,entrance,heated,place,addr_postc,website,historic,type,toilets_ch,Bing,surface,water_sour,public,artist_nam,start_date,payment_ca,payment_cr,seasonal,changing_1,levels,source_geo,bin,ele_msl,ele_source,source_pos,tactile_pa,phone,service_bi,service__1,service__2,service__3,service__4,toilets,internet_a

### Woa. Cambrian explosion of Features.

&nbsp;

#### Let's only keep a reduced set for today's analysis :). 

In [6]:
    gdf_amenity=gdf_amenity[['geometry', 'amenity', 'water' ,'drinking_w','wheelchair', 'wheelcha_1', 
                             'backrest', 'fee', 'name', 'geocaching','note']]
    gdf_amenity[:2]

,geometry,amenity,wheelchair,wheelcha_1,backrest,fee,name,geocaching,note
0,POINT (14.21335 56.59857),toilets,None,None,None,None,None,None,None
1,POINT (18.05524 59.33863),toilets,None,None,None,None,None,None,None


&nbsp;
# Now lets have a look at this features in a map

&nbsp;

#### We will use PyDeck for Jupyter (Deck's Jupyter plugin)

&nbsp;

**To enable pydeck for Jupyter, run this in command before starting the notebook:**

jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck

jupyter nbextension enable --sys-prefix --py pydeck

&nbsp;

##### For more info and examples, [check here](https://github.com/visgl/deck.gl/blob/master/bindings/pydeck/examples/geopandas_integration.py)


In [7]:
#create separate lng and lat facilitates plotting different things
gdf_amenity['lng']= list(map(lambda p: p.x, gdf_amenity.geometry ))
gdf_amenity['lat']= list(map(lambda p: p.y, gdf_amenity.geometry ))

## Now, let's prepare the layers

&nbsp;

Say we want to start with simple points in different colours (see docs [here](https://deck.gl/docs/api-reference/layers/scatterplot-layer)).

In [93]:
opacity=0.5
radius=40
kwargs={ 'pickable':True,  'get_position':['lng', 'lat'],  'opacity':opacity}
toilet_point_ly = pdk.Layer("ScatterplotLayer", getFillColor=[237,102,99],#ed6663
                            data=gdf_amenity.loc[gdf_amenity.amenity=='toilets'],
                            getRadius=radius*1.5, **kwargs) 
bench_point_ly = pdk.Layer("ScatterplotLayer", getFillColor=[78,137,174],#4e89ae
                           data=gdf_amenity.loc[gdf_amenity.amenity=='bench'],
                            getRadius=radius, **kwargs)

### And now, we also want some way to aggregate points

&nbsp;
#### We can try using Hexagon heatmaps from H3
There are a couple of ways to do that, today we will use the simplest Hexagon layer.

&nbsp;
##### ...Then, we need to do one more thing: 
&nbsp;
That is assigning H3 ids to each point.

&nbsp;
##### Trick question: Which resolutions do we want?
For observational purposes I chose the following cases:

| Resolution | Hex Edge Km  Avg|
| --- | --- |
| 8 | 0.46 |
| 6 | 3.23 |
| 4 | 22.61 |

For more on resolutions, see [this link](https://h3geo.org/docs/core-library/restable).

### To colour the hexes, we might want to use the count of points belonging to that hexagon. 

&nbsp;

#### This is how we construct such dataframe

In [94]:
for resolution in (4,6,8):
    gdf_amenity[f'h3id_{resolution}']=[h3.geo_to_h3(gdf_amenity.lat[x],
                                                                                gdf_amenity.lng[x],resolution) 
                                                                          for x in gdf_amenity.index]

In [95]:
cnt_dfs_bench={} #and we store a dataframe per resolution
cnt_dfs_toilet={}
for res in (4,6,8):
    cnt_dfs_bench[res]=gdf_amenity.loc[gdf_amenity.amenity=='bench'] [f'h3id_{res}']\
               .value_counts().to_frame()\
               .rename(columns={f'h3id_{res}':f'cnt'}).reset_index()\
               .rename(columns={'index':f'h3id_{res}'})
    cnt_dfs_toilet[res]=gdf_amenity.loc[gdf_amenity.amenity=='toilets'][f'h3id_{res}']\
                .value_counts().to_frame()\
                .rename(columns={f'h3id_{res}':f'cnt'}).reset_index()\
                .rename(columns={'index':f'h3id_{res}'})
    #feature scale normalization
    mn=cnt_dfs_bench[res].cnt.min()
    dffx=cnt_dfs_bench[res].cnt.max()-mn
    cnt_dfs_bench[res]['norm']=cnt_dfs_bench[res].cnt.apply(lambda x: (x-mn)/dffx)
    mn=cnt_dfs_toilet[res].cnt.min()
    dffx=cnt_dfs_toilet[res].cnt.max()-mn
    cnt_dfs_toilet[res]['norm']=cnt_dfs_toilet[res].cnt.apply(lambda x: (x-mn)/dffx)
cnt_dfs_toilet[4][:3] #for resolution 4, show first 3 lines

,h3id_4,cnt,norm
0,8408867ffffffff,246,1.000000
1,841f05bffffffff,77,0.310204
2,841f251ffffffff,70,0.281633


##### (Optional) Let's generate a viewport pointing somewhere that makes sense in the map

In [104]:
view_state = pdk.ViewState(latitude=57.71, longitude=11.94, zoom=6, bearing=0, 
                           pitch=0)

## Now we create our Hex layers :)

&nbsp;

For the sake of simplicity, let's look at the biggest resolution first (4)

&nbsp;

Check out the docs for Deck's  Hexagon Layer [here](https://pydeck.gl/gallery/h3_hexagon_layer.html)

In [212]:
resolution=6 #4,6,8
kwargs={ 'get_line_color':'[50,130,184]', 'line_width_min_pixels':0.3, 'pickable':True, 
                   'filled':True, 'stroked':True, 'extruded':False, 'opacity':0.28,
                    'get_hexagon':f'h3id_{resolution}' }
toilet_hex_ly = pdk.Layer("H3HexagonLayer",  cnt_dfs_toilet[resolution], 
                          getFillColor="[78+10/(norm*count),160-100*(norm+0.2),4000*norm]",
                          **kwargs  ) 
bench_hex_ly = pdk.Layer("H3HexagonLayer", cnt_dfs_bench[resolution], 
                           getFillColor="[78+10/(norm*count),160-100*(norm+0.2),4000*norm]",
                         **kwargs ) 

## And now let's draw our map!

Note: Be careful, playing with the fill colors **can** be a bit addictive.

In [213]:
ze_map=pdk.Deck(layers=[bench_point_ly,bench_hex_ly], map_style=map_style,
initial_view_state=view_state, mapbox_key=mapbox_key)

In [214]:
ze_map.show()

DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"bearing": 0, "latitude": 57.71, "longitu…

&nbsp;

# Thanks for watching / reading!

&nbsp;

&nbsp;

##### If you are curious about _what else_ is well documented in OpenStreetMap, check out [overpass-turbo](http://overpass-turbo.eu/) and the [overpass python wrapper](https://github.com/mvexel/overpass-api-python-wrapper/tree/master/examples).

&nbsp;